##### Copyright 2019 The TensorFlow Neural Structured Learning Authors

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Regularização adversária para classificação de imagens

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

Neste tutorial, exploraremos o uso do aprendizado adversário ([Goodfellow et al., 2014](https://arxiv.org/abs/1412.6572)) para classificação de imagens usando o framework Neural Structured Learning (NSL).

A ideia central do aprendizado adversário é treinar um modelo com dados adversariamente perturbados (chamados de exemplos adversários), além dos dados de treinamento orgânico. Para o olho humano, esses exemplos adversários parecem iguais aos originais, mas a perturbação fará com que o modelo fique confuso e faça previsões ou classificações incorretas. Os exemplos adversários são construídos para enganar intencionalmente o modelo, fazendo-o com que ele faça previsões ou classificações incorretas. Ao treinar com esses exemplos, o modelo aprende a ser robusto contra perturbações adversárias ao fazer previsões.

Neste tutorial, ilustraremos o seguinte procedimento de aplicação de aprendizado adversário para obter modelos robustos usando o framework Neural Structured Learning:

1. Crie uma rede neural como modelo de referência. Neste tutorial, o modelo de referência é criado com a API funcional `tf.keras`; este procedimento também é compatível com modelos criados por APIs sequenciais e de subclasse `tf.keras`. Para mais informações sobre os modelos Keras no TensorFlow, consulte esta [documentação](https://www.tensorflow.org/api_docs/python/tf/keras/Model) .
2. Envolva o modelo de referência com a classe wrapper **`AdversarialRegularization`**, que é fornecida pelo framework NSL, para criar uma nova instância de `tf.keras.Model`. Este novo modelo incluirá a perda adversária como termo de regularização em seu objetivo de treinamento.
3. Converta exemplos nos dados de treinamento em dicionários de características.
4. Treine e avalie o novo modelo.

## Recapitulação para iniciantes

Há uma [explicação em vídeo](https://youtu.be/Js2WJkhdU7k) relacionada sobre aprendizado adversário para classificação de imagens, parte da série TensorFlow Neural Structured Learning do YouTube. Abaixo, resumimos os principais conceitos explicados neste vídeo, expandindo a explicação fornecida na seção Visão Geral acima.

O framework NSL otimiza em conjunto as características de imagem e os sinais estruturados para fazer com que as redes neurais aprendam melhor. Mas e se não houver uma estrutura explícita disponível para treinar a rede neural? Este tutorial explica uma abordagem que envolve a criação de vizinhos adversários (modificados a partir da amostra original) para construir uma estrutura dinamicamente.

Primeiro, vizinhos adversários são definidos como versões modificadas da imagem da amostra aplicada com pequenas perturbações que induzem uma rede neural a gerar classificações imprecisas. Essas perturbações cuidadosamente projetadas são normalmente baseadas na direção reversa do gradiente e têm como objetivo confundir a rede neural durante o treinamento. Os humanos podem não ser capazes de distinguir entre uma imagem de amostra e seu vizinho adversário gerado. No entanto, para a rede neural, as perturbações aplicadas são eficazes em levar a uma conclusão imprecisa.

Os vizinhos adversários gerados são então conectados à amostra, construindo assim dinamicamente uma estrutura borda por borda. Usando essa conexão, as redes neurais aprendem a manter as semelhanças entre a amostra e os vizinhos adversários, evitando confusões resultantes de classificações incorretas, melhorando assim a qualidade e a precisão geral da rede neural.

O segmento de código abaixo é uma explicação em alto nível das etapas envolvidas, enquanto o restante deste tutorial vai mais longe e se aprofunda nos detalhes técnicos.

1. Leia e prepare os dados. Carregue o dataset MNIST e normalize os valores do características para que permaneçam no intervalo [0,1]

```
import neural_structured_learning as nsl

(x_train, y_train), (x_train, y_train) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
```

1. Construa a rede neural. Um modelo de referência Keras Sequencial é usado para este exemplo.

```
model = tf.keras.Sequential(...)
```


1. Configure o modelo adversário. Incluindo os hiperparâmetros: multiplicador aplicado na regularização adversária, valores diferentes escolhidos empiricamente para tamanho do passo/taxa de aprendizagem. Invoque a regularização adversária com uma classe wrapper em torno da rede neural construída.

```
adv_config = nsl.configs.make_adv_reg_config(multiplier=0.2, adv_step_size=0.05)
adv_model = nsl.keras.AdversarialRegularization(model, adv_config)
```

1. Conclua com o workflow padrão do Keras: compilar, ajustar, avaliar.

```
adv_model.compile(optimizer='adam', loss='sparse_categorizal_crossentropy', metrics=['accuracy'])
adv_model.fit({'feature': x_train, 'label': y_train}, epochs=5)
adv_model.evaluate({'feature': x_test, 'label': y_test})
```

O que você viu aqui foi o aprendizado adversário habilitado em 2 etapas e com 3 linhas simples de código. Esta é a simplicidade do framework Neural Structured Learning. Nas seções a seguir, expandiremos esse procedimento.

## Configuração

Instale o pacote Neural Structured Learning.

In [ ]:
!pip install --quiet neural-structured-learning

Importe as bibliotecas. Abreviamos `neural_structured_learning` para `nsl`.

In [ ]:
import matplotlib.pyplot as plt
import neural_structured_learning as nsl
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Hiperparâmetros

Coletamos e explicamos os hiperparâmetros (num objeto `HParams`) para treinamento e avaliação do modelo.

Entrada/Saída:

- **`input_shape`**: o formato do tensor de entrada. Cada imagem tem 28 por 28 pixels com 1 canal.
- **`num_classes`**: há um total de 10 classes, correspondendo a 10 dígitos [0-9].

Arquitetura do modelo:

- **`conv_filters`**: uma lista de números, cada um especificando o número de filtros em uma camada convolucional.
- **`kernel_size`**: o tamanho da janela de convolução 2D, compartilhada por todas as camadas convolucionais.
- **`pool_size`**: fatores para reduzir a escala da imagem em cada camada de pooling máximo.
- **`num_fc_units`**: o número de unidades (ou seja, largura) de cada camada totalmente conectada.

Treinamento e avaliação:

- **`batch_size`**: tamanho do lote usado para treinamento e avaliação.
- **`epochs`**: o número de épocas de treinamento.

Aprendizagem adversária:

- **`adv_multiplier`**: o peso da perda adversária no objetivo de treinamento, em relação à perda rotulada.
- **`adv_step_size`**: a magnitude da perturbação adversária.
- **`adv_grad_norm`**: a norma para medir a magnitude da perturbação adversária.


In [ ]:
class HParams(object):
  def __init__(self):
    self.input_shape = [28, 28, 1]
    self.num_classes = 10
    self.conv_filters = [32, 64, 64]
    self.kernel_size = (3, 3)
    self.pool_size = (2, 2)
    self.num_fc_units = [64]
    self.batch_size = 32
    self.epochs = 5
    self.adv_multiplier = 0.2
    self.adv_step_size = 0.2
    self.adv_grad_norm = 'infinity'

HPARAMS = HParams()

## O dataset MNIST

O [dataset MNIST](http://yann.lecun.com/exdb/mnist/) contém imagens em escala de cinza de dígitos manuscritos (de '0' a '9'). Cada imagem mostra um dígito em baixa resolução (28 por 28 pixels). A tarefa é classificar as imagens em 10 categorias, uma por dígito.

Aqui carregamos o dataset MNIST de [TensorFlow Datasets](https://www.tensorflow.org/datasets). Ele cuida do download dos dados e da construção de um `tf.data.Dataset`. O dataset carregado possui dois subconjuntos:

- `train` com 60.000 exemplos e
- `test` com 10.000 exemplos.

Os exemplos em ambos os subconjuntos são armazenados em dicionários de características com as duas chaves a seguir:

- `image`: array de valores de pixels, variando de 0 a 255.
- `label`: rótulo de verdade absoluta (ground truth), variando de 0 a 9.

In [ ]:
datasets = tfds.load('mnist')

train_dataset = datasets['train']
test_dataset = datasets['test']

IMAGE_INPUT_NAME = 'image'
LABEL_INPUT_NAME = 'label'

Para deixar o modelo numericamente estável, normalizamos os valores dos pixels para [0, 1] mapeando o dataset sobre a função `normalize`. Depois de embaralhar o dataset de treinamento e o lote, convertemos os exemplos em tuplas de características `(image, label)` para treinar o modelo de referência. Também fornecemos uma função para converter tuplas em dicionários para uso posterior.

In [ ]:
def normalize(features):
  features[IMAGE_INPUT_NAME] = tf.cast(
      features[IMAGE_INPUT_NAME], dtype=tf.float32) / 255.0
  return features

def convert_to_tuples(features):
  return features[IMAGE_INPUT_NAME], features[LABEL_INPUT_NAME]

def convert_to_dictionaries(image, label):
  return {IMAGE_INPUT_NAME: image, LABEL_INPUT_NAME: label}

train_dataset = train_dataset.map(normalize).shuffle(10000).batch(HPARAMS.batch_size).map(convert_to_tuples)
test_dataset = test_dataset.map(normalize).batch(HPARAMS.batch_size).map(convert_to_tuples)

## Modelo de referência

Nosso modelo de referência será uma rede neural composta por 3 camadas convolucionais seguidas por 2 camadas totalmente conectadas (conforme definido em `HPARAMS`). Aqui nós o definimos usando a API funcional Keras. Sinta-se à vontade para experimentar outras APIs ou arquiteturas de modelo (por exemplo, subclasses). Observe que o framework NSL oferece suporte a todos os três tipos de APIs Keras.

In [ ]:
def build_base_model(hparams):
  """Builds a model according to the architecture defined in `hparams`."""
  inputs = tf.keras.Input(
      shape=hparams.input_shape, dtype=tf.float32, name=IMAGE_INPUT_NAME)

  x = inputs
  for i, num_filters in enumerate(hparams.conv_filters):
    x = tf.keras.layers.Conv2D(
        num_filters, hparams.kernel_size, activation='relu')(
            x)
    if i < len(hparams.conv_filters) - 1:
      # max pooling between convolutional layers
      x = tf.keras.layers.MaxPooling2D(hparams.pool_size)(x)
  x = tf.keras.layers.Flatten()(x)
  for num_units in hparams.num_fc_units:
    x = tf.keras.layers.Dense(num_units, activation='relu')(x)
  pred = tf.keras.layers.Dense(hparams.num_classes)(x)
  model = tf.keras.Model(inputs=inputs, outputs=pred)
  return model

In [ ]:
base_model = build_base_model(HPARAMS)
base_model.summary()

Em seguida, treinamos e avaliamos o modelo de referência.

In [ ]:
base_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc'])
base_model.fit(train_dataset, epochs=HPARAMS.epochs)

In [ ]:
results = base_model.evaluate(test_dataset)
named_results = dict(zip(base_model.metrics_names, results))
print('\naccuracy:', named_results['acc'])

Podemos ver que o modelo de referência atinge 99% de exatidão no dataset de teste. Veremos quão robusto ele é em [Robustez sob perturbações adversárias](#scrollTo=HXK9MGG8lBX3) abaixo.

## Modelo regularizado adversário

Aqui mostramos como incorporar o treinamento adversário num modelo Keras com algumas linhas de código, usando o framework NSL. O modelo de referência é empacotado para criar um novo `tf.Keras.Model`, cujo objetivo de treinamento inclui regularização adversária.

Primeiro, criamos um objeto de configuração com todos os hiperparâmetros relevantes usando a função auxiliar `nsl.configs.make_adv_reg_config`.

In [ ]:
adv_config = nsl.configs.make_adv_reg_config(
    multiplier=HPARAMS.adv_multiplier,
    adv_step_size=HPARAMS.adv_step_size,
    adv_grad_norm=HPARAMS.adv_grad_norm
)

Agora podemos encapsular um modelo de referência com `AdversarialRegularization`. Aqui criamos um novo modelo de referência (`base_adv_model`), para que o existente (`base_model`) possa ser usado em comparações posteriores.

O `adv_model` retornado é um objeto `tf.keras.Model`, cujo objetivo de treinamento inclui um termo de regularização para a perda adversária. Para calcular essa perda, o modelo precisa ter acesso às informações do rótulo (feature `label`), além da entrada regular (feature `image`). Por esse motivo, convertemos os exemplos de tuplas nos datasets de volta para dicionários. E informamos ao modelo qual característica contém as informações do rótulo por meio do parâmetro `label_keys`.

In [ ]:
base_adv_model = build_base_model(HPARAMS)
adv_model = nsl.keras.AdversarialRegularization(
    base_adv_model,
    label_keys=[LABEL_INPUT_NAME],
    adv_config=adv_config
)

train_set_for_adv_model = train_dataset.map(convert_to_dictionaries)
test_set_for_adv_model = test_dataset.map(convert_to_dictionaries)

Em seguida, compilamos, treinamos e avaliamos o modelo regularizado adversário. Pode haver avisos como "Output missing from loss dictionary", o que não tem problema pois o `adv_model` não depende da implementação de referência para calcular a perda total.

In [ ]:
adv_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc'])
adv_model.fit(train_set_for_adv_model, epochs=HPARAMS.epochs)

In [ ]:
results = adv_model.evaluate(test_set_for_adv_model)
named_results = dict(zip(adv_model.metrics_names, results))
print('\naccuracy:', named_results['sparse_categorical_accuracy'])

Podemos ver que o modelo regularizado adversário também tem um desempenho muito bom (99% de exatidão) no dataset de testes.

## Robustez sob perturbações adversárias

Agora comparamos o modelo de referência com o modelo regulamentado pelo adversário quanto à robustez sob perturbação adversária.

Usaremos a função `AdversarialRegularization.perturb_on_batch` para gerar exemplos adversariamente perturbados. E gostaríamos que a geração fosse baseada no modelo de referência. Para fazer isso, encapsulamos o modelo de referência com `AdversarialRegularization`. Observe que, desde que o treinamento (`Model.fit`) não seja invocado, as variáveis ​​aprendidas no modelo não mudarão e o modelo ainda será o mesmo da seção [Modelo de referência](#scrollTo=JrrMpPNmpCKK).

In [ ]:
reference_model = nsl.keras.AdversarialRegularization(
    base_model, label_keys=[LABEL_INPUT_NAME], adv_config=adv_config)
reference_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc'])

Juntamos num dicionário os modelos a serem avaliados, e também criamos um objeto de métricas para cada um dos modelos.

Observe que consideramos que `adv_model.base_model` tem o mesmo formato de entrada (sem exigir informações de rótulo) do modelo de referência. As variáveis ​​aprendidas em `adv_model.base_model` são as mesmas que em `adv_model`.

In [ ]:
models_to_eval = {
    'base': base_model,
    'adv-regularized': adv_model.base_model
}
metrics = {
    name: tf.keras.metrics.SparseCategoricalAccuracy()
    for name in models_to_eval.keys()
}

Aqui está o loop para gerar exemplos perturbados e avaliar modelos com eles. Salvamos as imagens, rótulos e previsões perturbadas para visualização na próxima seção.

In [ ]:
perturbed_images, labels, predictions = [], [], []

for batch in test_set_for_adv_model:
  perturbed_batch = reference_model.perturb_on_batch(batch)
  # Clipping makes perturbed examples have the same range as regular ones.
  perturbed_batch[IMAGE_INPUT_NAME] = tf.clip_by_value(
      perturbed_batch[IMAGE_INPUT_NAME], 0.0, 1.0)
  y_true = perturbed_batch.pop(LABEL_INPUT_NAME)
  perturbed_images.append(perturbed_batch[IMAGE_INPUT_NAME].numpy())
  labels.append(y_true.numpy())
  predictions.append({})
  for name, model in models_to_eval.items():
    y_pred = model(perturbed_batch)
    metrics[name](y_true, y_pred)
    predictions[-1][name] = tf.argmax(y_pred, axis=-1).numpy()

for name, metric in metrics.items():
  print('%s model accuracy: %f' % (name, metric.result().numpy()))

Podemos ver que a exatidão do modelo de referência cai drasticamente (de 99% para cerca de 50%) quando a entrada é perturbada de forma adversária. Por outro lado, a precisão do modelo regularizado adversário apenas degrada um pouco (de 99% para 95%). Isto mostrs a eficácia do aprendizado adversário na melhoria da robustez do modelo.

## Exemplos de imagens perturbadas adversariamente

Aqui damos uma olhada nas imagens perturbadas adversariamente. Podemos ver que as imagens perturbadas ainda mostram dígitos reconhecíveis por humanos, mas podem enganar com sucesso o modelo de referência.

In [ ]:
batch_index = 0

batch_image = perturbed_images[batch_index]
batch_label = labels[batch_index]
batch_pred = predictions[batch_index]

batch_size = HPARAMS.batch_size
n_col = 4
n_row = (batch_size + n_col - 1) // n_col

print('accuracy in batch %d:' % batch_index)
for name, pred in batch_pred.items():
  print('%s model: %d / %d' % (name, np.sum(batch_label == pred), batch_size))

plt.figure(figsize=(15, 15))
for i, (image, y) in enumerate(zip(batch_image, batch_label)):
  y_base = batch_pred['base'][i]
  y_adv = batch_pred['adv-regularized'][i]
  plt.subplot(n_row, n_col, i+1)
  plt.title('true: %d, base: %d, adv: %d' % (y, y_base, y_adv))
  plt.imshow(tf.keras.utils.array_to_img(image), cmap='gray')
  plt.axis('off')

plt.show()

## Conclusão

Demonstramos o uso do aprendizado adversário para classificação de imagens usando o framework Neural Structured Learning (NSL). Incentivamos os usuários a experimentar diferentes configurações adversárias (em hiperparâmetros) e ver como elas afetam a robustez do modelo.